First, load the data,in this notebook we are using the smokoy mountain data as the example. And will use basic AE to get the transformation of the data and do prediction using the same models.

In [1]:
import sys
sys.path.append('../../../Code')
import loadData 
import RunML
import RunML_continue
import FS
import metric

2025-01-16 11:38:13.998496: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-16 11:38:14.000958: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-16 11:38:14.004450: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-16 11:38:14.013952: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737045494.030391 1996084 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737045494.03

In [2]:
import pandas as pd
import numpy as np
import random
from sklearn import svm
import pickle
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: 2.18.0
Num GPUs Available:  0


2025-01-16 11:38:16.256556: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


### 1. Load the data and data preprocess
After reading the data, convert the abundance matrix into relative abundance matrix; for the response variables, since we have multiple variables, organize them into an array with each variable as a column.

In [4]:
data,burn_label,un_label,duration_label,ASVs,df=loadData.loadSoilData("../data/count_table/merge_proportion_asv_smoky_moutain.csv")

In [5]:
data = FS.relative_abundance(data)

### 2.AE


In [6]:
data_train, data_test, y_train, y_test = RunML_continue.split_and_scale_data(data,burn_label)
print(data_train.shape)
print(data_test.shape)

(41, 1768)
(18, 1768)


In [7]:
AE_train, AE_test = RunML_continue.run_AE(data_train,data_test)

latent dimension: 100, activation:tanh, opt:adam,MAE:0.33744439482688904
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


In [8]:
data_AE = pd.concat([AE_train, AE_test], axis=0).to_numpy()

In [9]:
iter =30
cls = ["RF","SVM", "CatBoost","NB"]

In [10]:
data_subset = {"AllFeatures":data, 
               "AE": data_AE
              }

In [11]:
print(np.shape(data))
print(np.shape(data_AE))

(59, 1768)
(59, 100)


The  function will print out the accuracy and AUC for each dataset using each classifier, and also will return the y_actual, y_predict, y_predprob for future use.

In [12]:
targetLabel=burn_label
dict_cm = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

TBB Warning: The number of workers is currently limited to 0. The request for 47 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



             RF_Accuracy   RF_AUC   SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.780303    0.922917    0.746970    0.843750      0.845455        
AE            0.712121    0.557292    0.709091    0.616898      0.663636        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.887037     0.863636    0.801389  
AE             0.500926     0.642424    0.632870  


In [13]:
print(metric.metric_sum(dict_cm))

                      Accuracy  Precision   Recall   Specification     Mcc  
AllFeatures_RF        0.779661  1.000000   0.235294    1.000000     0.423886
AllFeatures_SVM       0.745763  0.571429   0.470588    0.857143     0.348896
AllFeatures_CatBoost  0.847458  1.000000   0.470588    1.000000     0.622530
AllFeatures_NB        0.864407  0.846154   0.647059    0.952381     0.655004
AE_RF                 0.711864  0.500000   0.176471    0.928571     0.157398
AE_SVM                0.711864  0.500000   0.294118    0.880952     0.211331
AE_CatBoost           0.661017  0.384615   0.294118    0.809524     0.113248
AE_NB                 0.644068  0.166667   0.058824    0.880952    -0.090241


In [14]:
targetLabel=un_label
dict_cm_un = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy   RF_AUC   SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.556061    0.702698    0.556061    0.424444      0.524242        
AE            0.424242    0.439683    0.457576    0.439206      0.507576        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.582540     0.610606    0.605000  
AE             0.515873     0.442424    0.304603  


In [15]:
targetLabel=duration_label
dict_cm_dur = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls,SMOTE=True)

             RF_Accuracy   RF_AUC   SVM_Accuracy   SVM_AUC  CatBoost_Accuracy  \
AllFeatures   0.590909    0.457857    0.507576    0.471607      0.542424        
AE            0.545455    0.343036    0.460606    0.377321      0.390909        

             CatBoost_AUC  NB_Accuracy   NB_AUC   
AllFeatures    0.436429     0.507576    0.516071  
AE             0.336607     0.356061    0.340714  
